In [1]:
import numpy as np
import networkx as nx

import scipy.optimize
import scipy.interpolate
import scipy.special

import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import random
import matplotlib.colors as mcolors

from relm.mechanisms import LaplaceMechanism

In [2]:
df = pd.DataFrame
df = pd.read_csv("/local/2014-03-03-am.csv", header='infer')

## Try undirected graph

In [6]:
ids = df["client"].unique()
graphs = []
n = 500
print("Number of clients: ", n)
for id in ids[:n]:
    one_client = df.query('client == @id')
    one_client=one_client.rename(columns = {'Unnamed: 0':'indx'})

    g = nx.empty_graph(0, nx.Graph()) #initialize an empty graph

    indeces = one_client['indx'].values.tolist()
    counter = 0

    for i in range(len(indeces)-1):
        a = one_client.loc[one_client['indx'] == indeces[i], 'AP']
        b = one_client.loc[one_client['indx'] == indeces[i+1], 'AP']
        g.add_edges_from(zip(a, b), label = id, node_size=70)
        counter += 1
    #pos=nx.get_node_attributes(G,'pos')
    #labels = nx.get_edge_attributes(G,'weight')
    graphs.append(g)
      
G = graphs[0]
nx.set_edge_attributes(G, 'red', 'color')
for i in range(1, len(graphs)):
    random_color = random.choice(list(mcolors.CSS4_COLORS.keys()))
    curr = graphs[i]
    nx.set_edge_attributes(curr, random_color, 'color')
    #edges = curr.edges()
    G = nx.compose(G,curr)

Number of clients:  500


In [7]:
n = G.number_of_nodes()
print("Number of nodes:", n)

Number of nodes: 735


In [8]:
def exact_count(G, h):
    """
    Compute the exact value of a query which is linear in the degree distribution of G
    
    Parameters:
        G: An undirected graph
        h: An array describing a query which is linear in the degree distribution of G
        
    Returns:
        The exact value of the query evaluated on G.
    """
    x = np.arange(len(h))
    h_fun = scipy.interpolate.interp1d(x, h)
    degree_histogram = np.array(nx.degree_histogram(G))
    exact_count = degree_histogram @ h_fun(np.arange(len(degree_histogram)))
    return exact_count

In [9]:
# Edge count
h_edge = np.arange(n+1) / 2.0
edge_count = exact_count(G, h_edge)

# Node count
h_node = np.ones(n+1)
node_count = exact_count(G, h_node)

# kstar count
k = 3
h_kstar = scipy.special.comb(np.arange(n+1), k)
kstar_count = exact_count(G, h_kstar)

In [10]:
def build_flow_graph(G, D):
    """
    Build a flow graph for G
    
    Parameters:
        G: An undirected graph
        D: The capacity for edges between nodes of G and
           the source/sink nodes in the flow graph
           
    Returns:
        A flow graph whose max flow yields an approximate query response
    """
    V_left = list(zip(["left"] * len(G), G.nodes()))
    V_right = list(zip(["right"] * len(G), G.nodes()))
    F = nx.DiGraph()
    F.add_nodes_from(V_left)
    F.add_nodes_from(V_right)
    F.add_nodes_from("st")
    F.add_weighted_edges_from([("s", vl, D) for vl in V_left], weight="capacity")
    F.add_weighted_edges_from([(vr, "t", D) for vr in V_right], weight="capacity")
    F.add_weighted_edges_from(
        [(("left", u), ("right", v), 1) for u, v in G.edges()], weight="capacity"
    )
    F.add_weighted_edges_from(
        [(("left", v), ("right", u), 1) for u, v in G.edges()], weight="capacity"
    )
    return F

In [11]:
D = n-1
F = build_flow_graph(G, D)
print(F.number_of_nodes())

1472


In [12]:
def bounded_degree_quasiflow(F, h, D):
    """
    Parameters:
        G: An undirected graph
        h: An array describing a query which is linear in the degree distribution of G
        D: A bound on the capacities in the flow graph derived from G
        
    Returns:
        The maximal value for \max_{f \in flows} \sum{v \in F} h(f(v))
    """
    nodes = list(F.nodes())
    edges = list(F.edges())
    adjacency = np.zeros((len(nodes), len(edges)))
    for j in range(len(edges)):
        i0 = nodes.index(edges[j][0])
        i1 = nodes.index(edges[j][1])
        adjacency[i0, j] = -1
        adjacency[i1, j] = 1

    capacities = np.array([F.edges[e]["capacity"] for e in F.edges()])
    x0 = np.random.random(capacities.size) * capacities
    mask = np.array([("s" in edge) for edge in edges])
    bounds = [(0, capacity) for capacity in capacities]
    constraint = scipy.optimize.LinearConstraint(adjacency[:-2], 0, 0)

    x = np.arange(D + 1)
    print(x.shape)
    print(h[:D+1].shape)
    h_fun = scipy.interpolate.interp1d(x, h[:D+1])
    f = lambda x, *args: -np.sum(h_fun(x[tuple(args[0])]))
    res = scipy.optimize.minimize(
        fun=f, x0=x0, args=[mask], bounds=bounds, constraints=[constraint]
    )
    return -res.fun

In [ ]:
# Edge count
bd_quasiflow_edge = bounded_degree_quasiflow(F, h_edge, D)

(735,)
(735,)


In [ ]:
# Node count
bd_quasiflow_node = bounded_degree_quasiflow(F, h_node, D)

In [ ]:
# kstar count
bd_quasiflow_kstar = bounded_degree_quasiflow(F, h_kstar, D)

In [ ]:
# Create a differentially private release mechanism
epsilon = 1.0

# Edge count
sensitivity_edge = np.max(h_edge[:(D+1)]) + np.max(h_edge[1:(D+1)] - h_edge[:D])
mechanism_edge = LaplaceMechanism(epsilon=epsilon, sensitivity=sensitivity_edge)
dp_edge_count = mechanism_edge.release(np.array([bd_quasiflow_edge]))

# Node count
sensitivity_node = np.max(h_node[:(D+1)]) + np.max(h_node[1:(D+1)] - h_node[:D])
mechanism_node = LaplaceMechanism(epsilon=epsilon, sensitivity=sensitivity_node)
dp_node_count = mechanism_node.release(np.array([bd_quasiflow_node]))

# kstar count
# Note: The sensitivity of the kstar count is much greater than that of the edge and node counts
sensitivity_kstar = np.max(h_kstar[:(D+1)]) + np.max(h_kstar[1:(D+1)] - h_kstar[:D])
mechanism_kstar = LaplaceMechanism(epsilon=epsilon, sensitivity=sensitivity_kstar)
dp_kstar_count = mechanism_kstar.release(np.array([bd_quasiflow_kstar]))

In [ ]:
# Edge count
print("Exact edge count = %f" % edge_count)
print("Approximate edge count = %f" % bd_quasiflow_edge)
print("Differentially private edge count = %f\n" % dp_edge_count)

# Node count
print("Exact node count = %f" % node_count)
print("Approximate node count = %f" % bd_quasiflow_node)
print("Differentially private node count = %f\n" % dp_node_count)

# kstar count
print("Exact %i-star count = %f" % (k, kstar_count))
print("Approximate %i-star count = %f" % (k, bd_quasiflow_kstar))
print("Differentially private %i-star count = %f\n" % (k, dp_kstar_count))